In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

class FifaSpider(scrapy.Spider):
    name = 'FIFAS'
    
    start_urls = ['https://www.futhead.com/players/?sort=-if_total']
    
    def parse(self, response):
        for row in response.xpath('//li/div/a'):
            yield {
                'name': row.xpath('span[@class="player-info"]/span[@class="player-name"]/text()').extract(),
                'rating': row.xpath('span[1]/span/text()').extract(),
                'pace': row.xpath('span[3]/span[2]/span[@class="value"]/text()').extract(),
                'shooting': row.xpath('span[3]/span[3]/span[@class="value"]/text()').extract(),
                'passing': row.xpath('span[3]/span[4]/span[@class="value"]/text()').extract(),
                'dribbling': row.xpath('span[3]/span[5]/span[@class="value"]/text()').extract(),
                'defense': row.xpath('span[3]/span[6]/span[@class="value"]/text()').extract(),
                'physical': row.xpath('span[3]/span[7]/span[@class="value"]/text()').extract()
            }
        #link for the next page
        next_page = response.xpath('//div[@class="pagination pull-right"]/a[3]/@href').extract_first()
        #print(response.xpath('//div[@class="pagination pull-right"]/a[3]/@href').extract_first())
        # a counter for the pagenumbers and acts as a stopping condition
        pagenum = int(re.findall(r'\d+', next_page)[0])
        
        #scrape the first 10 pages
        if next_page is not None and pagenum <11:
            next_page = response.urljoin(next_page)
            
            yield scrapy.Request(next_page, callback=self.parse)
            
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'fifa_data.json',  # Name our storage file.
    'LOG_ENABLED': False,          # Turn off logging for now.
    'ROBOTSTXT_OBEY': True,
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.96 Safari/537.36',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True
})

process.crawl(FifaSpider)
process.start()
print('Finished')

Finished


In [2]:
import pandas as pd

# Checking whether we got data from all 10 pages
FIFASdf=pd.read_json('fifa_data.json', orient='records')
print(FIFASdf.shape)
print(FIFASdf.head())

(51, 8)
  defense dribbling                 name  pace passing physical rating  \
0    [42]      [98]       [Lionel Messi]  [95]    [90]     [70]   [96]   
1    [46]      [95]  [Cristiano Ronaldo]  [95]    [86]     [87]   [96]   
2    [55]      [92]        [Luis Suárez]  [85]    [83]     [83]   [92]   
3    [37]      [86]    [Gonzalo Higuaín]  [83]    [74]     [79]   [89]   
4    [38]      [96]          [Neymar Jr]  [94]    [83]     [65]   [92]   

  shooting  
0     [94]  
1     [96]  
2     [92]  
3     [90]  
4     [88]  
